# Data Processing

The purpose of this notebook is to download data and process it. As a result we will get a dataset ready for further analyses and modeling.

# Table of Contents

[1. Imports & Environment Configuration](#Imports-&-Environment-Configuration)

# Imports & Environment Configuration

In [1]:
%load_ext autoreload

import matplotlib.pyplot as plt
import pandas

In [2]:
# let matplotlib plots be part of Jupyter Notebook
%matplotlib inline

# set up higher resolution for matplotlib plots
%config InlineBackend.figure_format = 'retina'

# Load data

In [3]:
# TODO: basically write code :)